# Military forces interactions graphs

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
March 2025

------

## Introduction

It is easy to make a simple [Rock-Paper-Scissors](https://en.wikipedia.org/wiki/Rock_paper_scissors) (RPS) game graph using the Raku package ["Graph"](https://raku.land/zef:antononcube/Graph), [AAp1]. Here is such a graph in which the arrow directions indicate which item (vertex) wins:

In [16]:
#%html
my $g0 = Graph.new(<🪨 ✂️ ✂️ 📄 📄 🪨>.Hash):d;
$g0.dot(:3graph-size, engine => 'neato'):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- ✂️ -->
 
 ✂️ 
 
 ✂️ 
 
<!-- 📄 -->
 
 📄 
 
 📄 
 
<!-- ✂️->📄 -->
 
 ✂️->📄 
 
 
 
<!-- 🪨 -->
 
 🪨 
 
 🪨 
 
<!-- 📄->🪨 -->
 
 📄->🪨 
 
 
 
<!-- 🪨->✂️ -->
 
 🪨->✂️

Easy, but now we want to:
- Add labels to the edges 
    - Like "🪨 crushes ✂️"
- Change the items/weapons
    - Snake-Centipede-Snail
- Make graphs for game _upgrades_
    - Rock-Paper-Scissors-Lizard-Spock
    - Rock-Paper-Scissors-Fire-Water
- Utilize Large Language Models (LLMs) 
    - In order to simplify the graph making

In this notebook (document) we show how to do all of the above points.

**Remark:** Interesting analogies of the presented graphs can be made with warfare graphs, [AAv1]. For example, the graph tanks-infantry-guerilla is analogous to RPS.

### TL;DR

- LLMs "know" the RPS game and its upgrades. 
- LLMs know how to (mostly, reliably) translate to emojis.
- The package "Graph" (via [Graphviz DOT](https://graphviz.org/doc/info/lang.html)) can produce SVG plots that are readily rendered in different environments.
  - And the graphs of hand-games like RPS look good.
- The class `Graph` have handy methods and attributes that make the creation and modification of graphs smooth(er).

------

## Setup

This notebook is a Raku-chatbook, hence, its Jupyter session pre-loads certain packages and LLM-personas.

In [3]:
# Preloaded in any chatbook
# use LLM::Functions;
# use LLM::Prompts;

# Preloaded in a user init file
# use Graph;

# For this concrete session
use Text::Emoji;

LLM configurations:

In [4]:
my $conf4o = llm-configuration('chat-gpt', model => 'gpt-4o', :4096max-tokens, temperature => 0.4);
my $conf4o-mini = llm-configuration('chat-gpt', model => 'gpt-4o-mini', :4096max-tokens, temperature => 0.4);

($conf4o, $conf4o-mini)».Hash».elems

(24 24)

In [5]:
my $background = '#1F1F1F';
my $engine = 'neato';

my %opts =
    :$background,     
    :6graph-size, 
    :1edge-width,
    :3edge-font-size,
    edge-color => 'LightSlateGray',
    node-width => 0.2, node-height => 0.2, 
    node-shape => 'circle', 
    :node-labels, 
    :8node-font-size,
    node-fill-color => '#1F1F1F',
    node-color => 'LightSlateGray',
    node-stroke-width => 0.6,
    arrow-size => 0.25,
    :$engine;

my %opts-plain = merge-hash(%opts, {:5node-font-size, node-shape => 'ellipse', node-width => 0.27, node-height => 0.15});

(%opts, %opts-plain)».elems

(15 15)

### Additional

In [6]:
sub game-table(Graph:D $g, Str:D :$link-value = '+', Str:D :$missing-value = '-') {
    cross-tabulate($g.edges(:dataset), <from>, <to>)
    ==> -> %h { %h.map({ $_.key => ($g.vertex-list Z=> $_.value{$g.vertex-list}).Hash }).Hash }()
    ==> to-dataset(:$missing-value)
    ==> -> %h { for $g.vertex-list { %h{$_}{$_} = ''}; %h }()
    ==> -> %h { $g.vertex-list.map({ [|%h{$_}, "" => $_].Hash }) }()
    ==> to-html(field-names => ["", |$g.vertex-list])
    ==> { .Str.subst('1', $link-value, :g).subst('(Any)', $missing-value, :g) }()
}

&game-table

-----

## Convenient LLM functions

### Graph edges

Instead of using chat-cells, we can define an LLM function that provides the graph edges dataset for different RPS variants. Here is such an LLM function using "LLM::Functions", [AAp1], and "LLM::Prompts", [AAv2]:

In [11]:
my sub rps-edge-dataset($description, Str:D $game-name = 'Rock-Paper-Scissors', *%args) {
    llm-synthesize([
        "Give the edges the graph for this $game-name variant description",
        'Give the edges as an array of dictionaries. Each dictionary with keys "from", "to", "label",',
        'where "label" has the action of "from" over "to".',
        $description,
        llm-prompt('NothingElse')('JSON')
        ], 
        e => %args<llm-evaluator> // %args<e> // %args<conf> // $conf4o-mini,
        form => sub-parser('JSON'):drop
    )
}

&rps-edge-dataset

**Remark::** Both "LLM::Functions" and "LLM::Prompts" are pre-loaded in Raku chatbooks.

----

## Analogy with Rock-Paper-Scissors

First RPS analogy:

In [50]:
my $war-game = rps-edge-dataset('tanks attack infantry, guerillas defend against tanks, infantry attacks querillas')

[{from => Tanks, label => attack, to => Infantry} {from => Guerillas, label => defend, to => Tanks} {from => Infantry, label => attack, to => Guerillas}]

Comparison of RPS graphs:

In [51]:
#% html
my %edge-labels = Empty; 
for |$war-game -> %r { %edge-labels{%r<from>}{%r<to>} = %r<label> };
Graph.new($war-game, :directed).dot(|%opts-plain, :%edge-labels, :svg)
~
$g0.dot(|%opts, edge-labels => %edge-labels-emo, :svg)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Guerillas -->
 
 Guerillas 
 
 Guerillas 
 
<!-- Tanks -->
 
 Tanks 
 
 Tanks 
 
<!-- Guerillas->Tanks -->
 
 Guerillas->Tanks 
 
 
 defend 
 
<!-- Infantry -->
 
 Infantry 
 
 Infantry 
 
<!-- Infantry->Guerillas -->
 
 Infantry->Guerillas 
 
 
 attack 
 
<!-- Tanks->Infantry -->
 
 Tanks->Infantry 
 
 
 attack 
 
 
 
<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- ✂️ -->
 
 ✂️ 
 
 ✂️ 
 
<!-- 📄 -->
 
 📄 
 
 📄 
 
<!-- ✂️->📄 -->
 
 ✂️->📄 
 
 
 cuts 
 
<!-- 🪨 -->
 
 🪨 
 
 🪨 
 
<!-- 📄->🪨 -->
 
 📄->🪨 
 
 
 covers 
 
<!-- 🪨->✂️ -->
 
 🪨->✂️ 
 
 
 crushes

------

## Military forces interaction

Her is a graph edges LLM-generation based on the [Mermaid-JS](https://mermaid.js.org) code of a somewhat more involved diagram, [NM1]:

In [13]:
#% html
my $mmd-descr = q:to/END/;
graph TD
AT[Anti-tank weapons] --> |defend|Arm[Armor]
Arm --> |attack|IA[Infantry and Artillery] 
Air[Air force] --> |attack|Arm
Air --> |attack|IA
M[Missiles] --> |defend|Air
IA --> |attack|M
IA --> |attack|AT
END

my $war-game2 = rps-edge-dataset($mmd-descr);

$war-game2 ==> to-html(field-names => <from label to>)

from,label,to
Anti-tank weapons,defend,Armor
Armor,attack,Infantry and Artillery
Air force,attack,Armor
Air force,attack,Infantry and Artillery
Missiles,defend,Air force
Infantry and Artillery,attack,Missiles
Infantry and Artillery,attack,Anti-tank weapons


In [1]:
my $war-game2 = $[{:from("Anti-tank weapons"), :label("defend"), :to("Armor")}, {:from("Armor"), :label("attack"), :to("Infantry and Artillery")}, {:from("Air force"), :label("attack"), :to("Armor")}, {:from("Air force"), :label("attack"), :to("Infantry and Artillery")}, {:from("Missiles"), :label("defend"), :to("Air force")}, {:from("Infantry and Artillery"), :label("attack"), :to("Missiles")}, {:from("Infantry and Artillery"), :label("attack"), :to("Anti-tank weapons")}];

[{from => Anti-tank weapons, label => defend, to => Armor} {from => Armor, label => attack, to => Infantry and Artillery} {from => Air force, label => attack, to => Armor} {from => Air force, label => attack, to => Infantry and Artillery} {from => Missiles, label => defend, to => Air force} {from => Infantry and Artillery, label => attack, to => Missiles} {from => Infantry and Artillery, label => attack, to => Anti-tank weapons}]

The diagram does not correspond to modern warfare -- it is taken from doctoral thesis, [NM1], discussing reconstruction of historical military data.
The corresponding graph can be upgraded with drones in a similar way as Chuck-Norris-defeats-all upgrade above.

In [2]:
my $war-forces = Graph.new($war-game2, :directed); 
my $drone = "Air drones";
my $war-game-d = $war-game2.clone.append( $war-forces.vertex-list.map({ %( from => $drone, to => $_, label => 'attack' ) }) );
$war-game-d .= append( ['Missiles', 'Air force'].map({ %(from => $_, to => $drone, label => 'defend') }) );
my $war-forces-d = Graph.new($war-game-d, :directed);

Graph(vertexes => 6, edges => 14, directed => True)

Here is the corresponding table:

In [7]:
#% html
game-table($war-forces-d, link-value => '⊙', missing-value => '')

,Air drones,Air force,Anti-tank weapons,Armor,Infantry and Artillery,Missiles
Air drones,,⊙,⊙,⊙,⊙,⊙
Air force,⊙,,,⊙,⊙,
Anti-tank weapons,,,,⊙,,
Armor,,,,,⊙,
Infantry and Artillery,,,⊙,,,⊙
Missiles,⊙,⊙,,,,


Here is the graph with different coloring for "attack" edges (gray) and "defend" edges (blue):

In [11]:
#% html
$war-forces-d.vertex-coordinates = ($war-forces-d.vertex-list Z=> Graph::Cycle($war-forces-d.vertex-count).vertex-coordinates{^$war-forces-d.vertex-count}.values).Hash;

my %edge-labels;
$war-game-d.map({ %edge-labels{$_<from>}{$_<to>} = $_<label> });

my %highlight = 
    'SlateBlue' => Graph.new( $war-game-d.grep(*<label> eq 'defend'), :directed).edges;

$war-forces-d.dot(
    :%highlight,
    |merge-hash(%opts-plain, {:9graph-size, node-width => 0.7}),
    :!edge-labels, 
    :svg
)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Anti-tank weapons -->
 
 Anti-tank weapons 
 
 Anti-tank weapons 
 
<!-- Armor -->
 
 Armor 
 
 Armor 
 
<!-- Anti-tank weapons->Armor -->
 
 Anti-tank weapons->Armor 
 
 
 
<!-- Missiles -->
 
 Missiles 
 
 Missiles 
 
<!-- Air force -->
 
 Air force 
 
 Air force 
 
<!-- Missiles->Air force -->
 
 Missiles->Air force 
 
 
 
<!-- Air drones -->
 
 Air drones 
 
 Air drones 
 
<!-- Missiles->Air drones -->
 
 Missiles->Air drones 
 
 
 
<!-- Infantry and Artillery -->
 
 Infantry and Artillery 
 
 Infantry and Artillery 
 
<!-- Infantry and Artillery->Anti-tank weapons -->
 
 Infantry and Artillery->Anti-tank weapons 
 
 
 
<!-- Infantry and Artillery->Missiles -->
 
 Infantry and Artillery->Missiles 
 
 
 
<!-- Armor->Infantry and Artillery -->
 
 Armor->Infantry and Artillery 
 
 
 
<!-- Air force->Infantry and Artillery -->
 
 Air force->Infantry and Artillery 
 
 
 
<!-- Air force->Armor -->
 
 Air force->Armor 
 
 
 
<!-- Air force->Air drones -->
 
 Air force->Air drones 
 
 
 
<!-- Air drones->Anti-tank weapons -->
 
 Air drones->Anti-tank weapons 
 
 
 
<!-- Air drones->Missiles -->
 
 Air drones->Missiles 
 
 
 
<!-- Air drones->Infantry and Artillery -->
 
 Air drones->Infantry and Artillery 
 
 
 
<!-- Air drones->Armor -->
 
 Air drones->Armor 
 
 
 
<!-- Air drones->Air force -->
 
 Air drones->Air force

**Remark:** The graph above is just an example -- real-life military forces interactions are more complicated.

-------

## Generalized antagonism

Following the article [SM1] we can make a graph for multiple conflicts (narrated exposition is given [AAv1]): 

In [10]:
#% html

my @multi-conflict-edges = 
    %(from=>1, to=>5, label=>'Neutrality',   :!directed), %(from=>1, to=>3, label=>'Commensalism', :directed),
    %(from=>1, to=>4, label=>'Commensalism', :directed),  %(from=>2, to=>1, label=>'Coercion',     :directed),
    %(from=>2, to=>3, label=>'Alliance',     :!directed), %(from=>2, to=>4, label=>'Guerilla war', :directed),
    %(from=>3, to=>4, label=>'Conflict',     :!directed), %(from=>5, to=>3, label=>'Avoidance',    :directed),
    %(from=>5, to=>4, label=>'Alliance',     :!directed), %(from=>5, to=>2, label=>'Adaptation',   :directed);

@multi-conflict-edges .= deepmap({ $_ ~~ Bool:D ?? $_ !! $_.Str });

my %edge-labels;
@multi-conflict-edges.map({ %edge-labels{$_<from>}{$_<to>} = $_<label> });

my $mc = Graph.new;
my @dir-edges;
for @multi-conflict-edges -> %e { 
    $mc.edge-add(%e<from>, %e<to>, :directed);
    if !%e<directed> {
        $mc.edge-add(%e<to>, %e<from>, :directed)
    }
}

$mc.vertex-coordinates = ($mc.vertex-list Z=> Graph::Cycle($mc.vertex-count).vertex-coordinates{^$mc.vertex-count}.values).Hash;

$mc.dot(|merge-hash(%opts, {node-shape => 'square', :4edge-font-size }), :%edge-labels, highlight => { RosyBrown => <1 3 4>, SlateBlue => <2 5> }, :mixed, :svg)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 4->5 -->
 
 4->5 
 
 Alliance 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 5->2 -->
 
 5->2 
 
 
 Adaptation 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 5->3 -->
 
 5->3 
 
 
 Avoidance 
 
<!-- 2->4 -->
 
 2->4 
 
 
 Guerilla war 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 2->1 -->
 
 2->1 
 
 
 Coercion 
 
<!-- 3->4 -->
 
 3->4 
 
 Conflict 
 
<!-- 3->2 -->
 
 3->2 
 
 Alliance 
 
<!-- 1->4 -->
 
 1->4 
 
 
 Commensalism 
 
<!-- 1->5 -->
 
 1->5 
 
 Neutrality 
 
<!-- 1->3 -->
 
 1->3 
 
 
 Commensalism

-----

## References

### Articles, books, theses

[AJ1] Archer Jones,
["The Art of War in Western World"](https://www.press.uillinois.edu/books/?id=p069666),
(2000),
[University of Illinois Press](https://www.press.uillinois.edu).
768 pages, ISBN-10: 0252069668, ISBN-13: 978-0252069666.

[SM1] Sergei Makarenko et al., 
["Обобщенная модель Ланчестера, формализующая конфликт нескольких сторон"](https://www.researchgate.net/publication/356424971_Obobsennaa_model_Lancestera_formalizuusaa_konflikt_neskolkih_storon_The_General_Lanchester_Model_Defining_Multilateral_Conflicts), 
[Eng. "The General Lanchester Model Defining Multilateral Conflicts"], 
(2021),
Automation of Control Processes № 2 (64), doi: 10.35752/1991-2927-2021-2-64-66-76.

[NM1] Николай В. Митюков,
["Математические модели и программные средства для реконструкции военно-исторических данных"](https://www.dissercat.com/content/matematicheskie-modeli-i-programmnye-sredstva-dlya-rekonstruktsii-voenno-istoricheskikh-dann),
(2009),
[disserCat](https://www.dissercat.com).

### Packages

[AAp1] Anton Antonov, 
[Graph Raku package](https://github.com/antononcube/Raku-Graph),
(2024-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov, 
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov, 
[LLM::Prompts Raku package](https://github.com/antononcube/Raku-LLM-Prompts),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov, 
[Jupyter::Chatbook Raku package](https://github.com/antononcube/Raku-Jupyter-Chatbook),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[EMp1] Elizabeth Mattijsen,
[Text::Emoji Raku package](https://github.com/lizmat/Text-Emoji),
(2024-2025),
[GitHub/lizmat](https://github.com/lizmat).

### Videos

[AAv1] Anton Antonov,
["Upgrading Epidemiological Models into War Models"](https://www.youtube.com/watch?v=852vMS_6Qaw),
(2024),
[YouTube/@WolframResearch](https://www.youtube.com/@WolframResearch).

[Wv1] Wozamil
["Rock Paper Scissors Lizard Spock (Extended Cut) ~ The Big Bang Theory ~"](https://www.youtube.com/watch?v=x5Q6-wMx-K8),
(2012),
[YouTube@Wozamil](https://www.youtube.com/@Wozamil).